In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))

#Load using keras.preprocessing
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

# Create the model
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#Train the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)
print("Saving Model:")
model.save('My_trained_model/my_model') 


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
0.0 0.99618727
Epoch 1/10
92/92 [==============================] - 93s 1s/step - loss: 1.3582 - accuracy: 0.4179 - val_loss: 1.0852 - val_accuracy: 0.5681
Epoch 2/10
92/92 [==============================] - 85s 923ms/step - loss: 1.0199 - accuracy: 0.6018 - val_loss: 1.0079 - val_accuracy: 0.5995
Epoch 3/10
92/92 [==============================] - 91s 986ms/step - loss: 0.8321 - accuracy: 0.6802 - val_loss: 1.0039 - val_accuracy: 0.6104
Epoch 4/10
92/92 [==============================] - 94s 1s/step - loss: 0.6280 - accuracy: 0.7759 - val_loss: 0.8584 - val_accuracy: 0.6689
Epoch 5/10
92/92 [==============================] - 88s 955ms/step - loss: 0.4669 - accuracy: 0.8314 - val_loss: 0.9371 - val_accuracy: 0.6635
Epoch 6/10
92/92 [==============================] - 86s 939ms/step - loss: 0.2797 - accuracy: 0.9091 - val_loss: 1.1399 - val_accur

In [3]:
img = keras.preprocessing.image.load_img(
    'd1.jpg', target_size=(img_height, img_width)
)

# convert image to tensor or numpy
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to roses with a 99.42 percent confidence.
